In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove.6B.100d.txt


<h1>Our Primary Objective is to Understand the Pure Performance of Different RNN Architectures on Raw Text Data</h1>

In [1]:
from torch.utils.data import DataLoader , Dataset
import torch
import torch.nn.functional as F
from torchmetrics import Accuracy, Precision, Recall, F1Score
from torchmetrics.classification import BinaryConfusionMatrix # Specifically for binary classification
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

import os
import shutil
from datasets import load_from_disk

from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict
import time

In [3]:
from collections import Counter
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as DEFAULT_STOP_WORDS

# Ensure SpaCy is loaded
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")


# Dataset

In [5]:



dataset_name = "stanfordnlp/imdb"

dataset = load_dataset(dataset_name)

train_dataset = dataset['train']
test_dataset = dataset['test']

full_dataset = concatenate_datasets([train_dataset, test_dataset])
print(len(full_dataset))


50000


# Train Test Validation Splits

To know the right ratio of splits and what method to use for train_test_split.. 

- Check whether dataset is balanced , imbalanced ? If it is imbalanced then we should use stratified splitting to ensure that each split (train, val, test) maintains the same class distribution as the original dataset.
- Since dataset is moderately size around 50000, (80,10,10) split works and gives around 5000 for testing and validation which are quite enough

In [5]:
 # 1. Extract all labels
labels = full_dataset['label']

# 2. Count the occurrences of each label
label_counts = Counter(labels)

print("Class Distribution:")
total_samples = len(full_dataset)

# 3. Print counts and percentages
for label, count in sorted(label_counts.items()):
    percentage = (count / total_samples) * 100
    print(f"  Label {label}: {count} samples ({percentage:.2f}%)")

# 4. Determine if it's balanced
# A common heuristic for "imbalanced" is if the ratio of the majority to minority class
# is significantly high (e.g., 2:1, 5:1, 10:1 or more).
# For simplicity, we'll just check if there's a large variance in percentages.
if len(label_counts) > 1:
    min_percentage = min(label_counts.values()) / total_samples * 100
    max_percentage = max(label_counts.values()) / total_samples * 100

    if max_percentage / min_percentage > 1.5: # Adjust this threshold as needed
        print("\nConclusion: The dataset appears to be imbalanced.")
        print(f"  Ratio of majority to minority class: {max_percentage:.2f}% / {min_percentage:.2f}% = {max_percentage / min_percentage:.2f}x")
    else:
        print("\nConclusion: The dataset appears to be relatively balanced.")
else:
    print("\nConclusion: Only one class found. The dataset has no imbalance (or is a single-class dataset).")

Class Distribution:
  Label 0: 25000 samples (50.00%)
  Label 1: 25000 samples (50.00%)

Conclusion: The dataset appears to be relatively balanced.


Since dataset is balanced, we can use normal splitting 

In [7]:
 #Step 1: Split into 90% (train+val) and 10% (test)

train_val_test_split = full_dataset.train_test_split(test_size=0.1, seed=42)

final_test_dataset = train_val_test_split['test']
train_val_dataset = train_val_test_split['train']

# Step 2: Split the 90% (train+val) into 8/9 for train and 1/9 for validation
train_val_split = train_val_dataset.train_test_split(test_size=(0.1/0.9), seed=42)

final_train_dataset = train_val_split['train']
final_validation_dataset = train_val_split['test'] # This is your validation set


# Combine into a DatasetDict for convenience
final_splits = DatasetDict({
    'train': final_train_dataset,
    'validation': final_validation_dataset,
    'test': final_test_dataset
})

print(f"Original dataset size: {len(full_dataset)}")
print(f"Train dataset size: {len(final_splits['train'])}")
print(f"Validation dataset size: {len(final_splits['validation'])}")
print(f"Test dataset size: {len(final_splits['test'])}")

# Verify the approximate ratios
total_samples = len(full_dataset)
print(f"Train ratio: {len(final_splits['train']) / total_samples:.2f}")
print(f"Validation ratio: {len(final_splits['validation']) / total_samples:.2f}")
print(f"Test ratio: {len(final_splits['test']) / total_samples:.2f}")










Original dataset size: 50000
Train dataset size: 40000
Validation dataset size: 5000
Test dataset size: 5000
Train ratio: 0.80
Validation ratio: 0.10
Test ratio: 0.10


In [9]:
###### A smaller dataset for convenience of training 





ratio = 1

num_train_samples = len(final_train_dataset)
small_train_size = int(num_train_samples * ratio)
small_train_dataset = final_train_dataset.select(range(small_train_size))


num_test_samples = len(final_test_dataset)
small_test_size = int(num_test_samples * ratio)
small_test_dataset = final_test_dataset.select(range(small_test_size))



num_validation_samples = len(final_validation_dataset)
small_validation_size = int(num_test_samples * ratio)
small_validation_dataset = final_test_dataset.select(range(small_test_size))


print(f"Small train dataset size : {len(small_train_dataset)}")
print(f"Small test dataset size : {len(small_test_dataset)}")
print(f"Small validation dataset size : {len(small_validation_dataset)}")


final_splits_reduced_dataset = DatasetDict({
    'train': small_train_dataset,
    'validation': small_validation_dataset,
    'test': small_test_dataset
})
final_splits_reduced_dataset

Small train dataset size : 40000
Small test dataset size : 5000
Small validation dataset size : 5000


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})


# Tokenizing and Creating Embeddings of Dataset using glove

**Contractions are Tokens:** For sentiment analysis, contractions (like "n't", "'s") often carry significant meaning (negation, possession). Do NOT filter them out. Ensure they are treated as individual tokens and that your pre-trained embedding vocabulary contains entries for them. GloVe usually has entries for common contractions.

**Consistency is Key:** Whatever tokenization and preprocessing rules you decide on (e.g., lowercasing, stop word removal, punctuation handling), apply them identically when building your vocabulary from the training data and when processing new text (validation/test sets).m

**Stop Words: A Decision:** Removing stop words (a, the, is, are) can reduce vocabulary size and noise, potentially speeding up training. However, sometimes they are crucial for sentiment (e.g., "this is not good"). For your first experiments, you might try with and without stop word removal to see the impact. SpaCy provides a good list of stop words.

### Customizing the tokenizing scheme for our task for sentiment classification

In [11]:


text = " <> I do not like 6  :)  this movie at all no 100. It was never truly bad, but also not amazing. No, I don't feel good about it."

print("--- Default SpaCy Stop Word List ---")
print(f"Contains 'not': {'not' in DEFAULT_STOP_WORDS}")
print(f"Contains 'no': {'no' in DEFAULT_STOP_WORDS}")
print(f"Contains 'never': {'never' in DEFAULT_STOP_WORDS}")
print(f"Contains 'do': {'do' in DEFAULT_STOP_WORDS}") # 'do' often is
print(f"Contains 'all': {'all' in DEFAULT_STOP_WORDS}") # 'at all' can be intensifying

# --- Step 1: Define Negation and Sentiment-Critical Words to KEEP ---
# This list can be expanded based on your domain and observation.
NEGATION_WORDS_TO_KEEP = {
    "not", "no", "never", "none", "nothing", "nowhere", "hardly", "scarcely", "barely",
    "don't", "doesn't", "didn't", "isn't", "aren't", "wasn't", "weren't",
    "won't", "wouldn't", "can't", "couldn't", "shouldn't", "mightn't", "mustn't"
}



# --- Step 2: Create a Custom Stop Word List ---
# Remove our negation words from the default SpaCy stop words
CUSTOM_STOP_WORDS = DEFAULT_STOP_WORDS - NEGATION_WORDS_TO_KEEP

print("\n--- Custom Stop Word List Check ---")
print(f"Custom stop words contain 'not': {'not' in CUSTOM_STOP_WORDS}")
print(f"Custom stop words contain 'no': {'no' in CUSTOM_STOP_WORDS}")
print(f"Custom stop words contain 'never': {'never' in CUSTOM_STOP_WORDS}")
print(f"Custom stop words contain 'do': {'do' in CUSTOM_STOP_WORDS}") # 'do' might still be there if not in NEGATION_WORDS_TO_KEEP

# --- Step 3: Apply Custom Stop Word List During Tokenization and Filtering ---
def custom_tokenize_and_filter(text_input, nlp_model, stop_words_set, include_emoticons=True):
    doc = nlp_model(text_input)
    processed_tokens = []
    for token in doc:
        token_text_lower = token.text.lower()
        
        # Rule 1: Filter out standalone punctuation and spaces
        if token.is_punct or token.is_space:
            # Rule 1a: Optional - keep specific emoticons even if they are punctuation
            if include_emoticons and token_text_lower in [":-)", ":)", ":(", ";)", ":P", ":O", ":|"]:
                processed_tokens.append(token_text_lower)
            continue # Skip other punctuation and spaces

        # Rule 2: Filter out stop words (but only if NOT in our kept negation list)
        if token_text_lower in stop_words_set:
            continue # Skip stop words

        # Rule 3: Keep numbers as is (or replace with [NUM] if preferred)
        # For sentiment, keeping them is often fine, unless they are just noise.
        if token.like_num:
           processed_tokens.append("<NUM>")
  
        else:
         processed_tokens.append(token_text_lower)
            
    return processed_tokens

print("\n--- Tokenization with Custom Stop Words ---")
tokens_processed = custom_tokenize_and_filter(text, nlp, CUSTOM_STOP_WORDS, include_emoticons=True)
print(f"Original: {text}")
print(f"Processed tokens: {tokens_processed}")

# Demonstrate with another example for clarity
text_positive = "This is really good, I absolutely love it!"
tokens_positive = custom_tokenize_and_filter(text_positive, nlp, CUSTOM_STOP_WORDS)
print(f"\nOriginal: {text_positive}")
print(f"Processed tokens: {tokens_positive}")

text_negative = "I hate this, it's not at all enjoyable. Never again."
tokens_negative = custom_tokenize_and_filter(text_negative, nlp, CUSTOM_STOP_WORDS)
print(f"\nOriginal: {text_negative}")
print(f"Processed tokens: {tokens_negative}")

# --- Vocabulary Building (Example using this custom function) ---
dummy_texts_large = [
    "This is a great movie no, I love it!",
    "The acting was terrible and the plot was boring.",
    "A wonderful experience, highly recommended.",
    "Not bad, but could be better.",
    "This movie is fantastic and amazing.",
    "The director's vision was inspiring and unique.  3 7",
    "Absolutely horrendous, a complete waste of time.",
    "An uplifting and heartwarming story.",
    "Could not stop laughing, truly hilarious.",
    "I don't like this film at all.6 9 "
] * 10 # Simulating more data

all_tokens_vocab_build = []
for t in dummy_texts_large:
    processed_sentence_tokens = custom_tokenize_and_filter(t, nlp, CUSTOM_STOP_WORDS)
    all_tokens_vocab_build.extend(processed_sentence_tokens)

token_counts_vocab_build = Counter(all_tokens_vocab_build)

PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'
word_to_idx = {PAD_TOKEN: 0, UNK_TOKEN: 1}
current_idx = 2
for word, count in token_counts_vocab_build.most_common(): # Use all words from your dataset for the vocab
    if word not in word_to_idx:
        word_to_idx[word] = current_idx
        current_idx += 1

VOCAB_SIZE_YOUR_DATA = len(word_to_idx)
print(f"\nYour dataset's vocabulary size after smarter processing with custom stopwords: {VOCAB_SIZE_YOUR_DATA}")
print(f"Word to index for 'not': {word_to_idx.get('not')}") # Should have an index
print(f"Word to index for 'no': {word_to_idx.get('no')}")   # Should have an index
print(f"Word to index for 'is': {word_to_idx.get('is')}")   # Should be None if removed



--- Default SpaCy Stop Word List ---
Contains 'not': True
Contains 'no': True
Contains 'never': True
Contains 'do': True
Contains 'all': True

--- Custom Stop Word List Check ---
Custom stop words contain 'not': False
Custom stop words contain 'no': False
Custom stop words contain 'never': False
Custom stop words contain 'do': True

--- Tokenization with Custom Stop Words ---
Original:  <> I do not like 6  :)  this movie at all no 100. It was never truly bad, but also not amazing. No, I don't feel good about it.
Processed tokens: ['<', '>', 'not', 'like', '<NUM>', ':)', 'movie', 'no', '<NUM>', 'never', 'truly', 'bad', 'not', 'amazing', 'no', 'feel', 'good']

Original: This is really good, I absolutely love it!
Processed tokens: ['good', 'absolutely', 'love']

Original: I hate this, it's not at all enjoyable. Never again.
Processed tokens: ['hate', 'not', 'enjoyable', 'never']

Your dataset's vocabulary size after smarter processing with custom stopwords: 39
Word to index for 'not': 4
W

In [14]:
small_train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 40000
})

###  Build Vocabulary and Determine MAX_LEN from TRAINING SPLIT ONLY ---

In [ ]:

def batch_tokenize_for_vocab(examples):
    all_tokens_for_batch_examples = []
    lengths_in_batch = []

    for text in examples['text']:
        tokens = custom_tokenize_and_filter(text, nlp, CUSTOM_STOP_WORDS)
        all_tokens_for_batch_examples.append(tokens)
        lengths_in_batch.append(len(tokens))

    return {"_temp_tokens": all_tokens_for_batch_examples, "_temp_lengths": lengths_in_batch}

# It's good practice to set num_proc to the number of CPU cores you want to use.
NUM_CPU_CORES = os.cpu_count()
print(f"Using {NUM_CPU_CORES} CPU cores for multiprocessing.")

# Assuming 'small_train_dataset' is already defined.
tokenized_train_data = small_train_dataset.map(
    batch_tokenize_for_vocab,
    batched=True,
    num_proc=NUM_CPU_CORES,
    remove_columns=small_train_dataset.column_names
)



Using 4 CPU cores for multiprocessing.


Map (num_proc=4):   0%|          | 0/40000 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset[1]

In [ ]:
tokenized_train_data[1]

In [ ]:
# --- Collect all tokens and lengths from the mapped dataset ---
all_train_tokens = []
train_lengths = []

for example in tokenized_train_data:

    all_train_tokens.extend(example['_temp_tokens'])

    # For _temp_lengths:
    current_lengths = example['_temp_lengths']
    if isinstance(current_lengths, int):
        # If it's a single int, wrap it in a list to make it iterable
        train_lengths.append(current_lengths) # Use append if it's a single item
    elif isinstance(current_lengths, list):
        # If it's a list, extend as usual
        train_lengths.extend(current_lengths)
    else:
        # Handle other unexpected types if necessary
        print(f"Warning: Unexpected type for _temp_lengths: {type(current_lengths)}")
        # Optionally, convert to list and extend, or raise an error
        train_lengths.extend(list(current_lengths)) # Try to convert to list

train_lengths

In [ ]:
#### Vocabulary Building 

token_counts = Counter(all_train_tokens)

# Define special tokens
PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'

# Build the word_to_idx mapping
word_to_idx = {PAD_TOKEN: 0, UNK_TOKEN: 1}
current_idx = 2
for word, count in token_counts.most_common(): # Iterates through words by frequency
    if word not in word_to_idx:
        word_to_idx[word] = current_idx
        current_idx += 1

idx_to_word = {idx: word for word, idx in word_to_idx.items()} # Useful for debugging

VOCAB_SIZE = len(word_to_idx)
print(f"Vocabulary size (from train set): {VOCAB_SIZE}")
print(f"'not' index: {word_to_idx.get('not', 'Not Found')}")
print(f"'no' index: {word_to_idx.get('no', 'Not Found')}")
print(f"'is' index: {word_to_idx.get('is', 'Not Found')}") # Should be 'Not Found'

# Determine MAX_LEN from training data
MAX_LEN = max(train_lengths)
# MAX_LEN = int(np.percentile(train_lengths, 99.5))

# Optional: Cap MAX_LEN to a reasonable percentile if you have extreme outliers
# If MAX_LEN is extremely high due to one very long sentence, it can cause excessive padding.
# You might do: MAX_LEN = int(np.percentile(train_lengths, 99))


print(f"Max sequence length (from train set): {MAX_LEN}")

# --- You would save word_to_idx and MAX_LEN here if this were a production pipeline ---


In [ ]:
# --- Saving word_to_idx and MAX_LEN ---
import json

output_dir = '/kaggle/working'
os.makedirs(output_dir, exist_ok=True) # Create the directory if it doesn't exist

# Save word_to_idx (as JSON)
word_to_idx_path = os.path.join(output_dir, 'word_to_idx.json')
with open(word_to_idx_path, 'w') as f:
    json.dump(word_to_idx, f)
print(f"Saved word_to_idx to: {word_to_idx_path}")

# Save MAX_LEN (as a simple text file)
max_len_path = os.path.join(output_dir, 'MAX_LEN.txt')
with open(max_len_path, 'w') as f:
    f.write(str(MAX_LEN))
print(f"Saved MAX_LEN to: {max_len_path}")



# Verify
print(f"Is loaded_word_to_idx the same as word_to_idx? {loaded_word_to_idx == word_to_idx}")
print(f"Is loaded_max_len the same as MAX_LEN? {loaded_max_len == MAX_LEN}")

In [15]:
import json

# --- How to load them later ---
print("\n--- Demonstrating how to load them later ---")


word_to_idx_path = "/kaggle/input/word-to-txt/word_to_idx.json"
# Load word_to_idx
with open(word_to_idx_path, 'r') as f:
    word_to_idx = json.load(f)
print(f"Loaded word_to_idx. Sample: {list(word_to_idx.items())[:5]}...")

MAX_LEN = 1337
VOCAB_SIZE = len(word_to_idx)


--- Demonstrating how to load them later ---
Loaded word_to_idx. Sample: [('<PAD>', 0), ('<UNK>', 1), ('/><br', 2), ('movie', 3), ('film', 4)]...


## Applying Preprocessing to All Splits

we'll create a single function that:

    - Takes a batch of examples from the dataset.

    - Applies custom_tokenize_and_filter to each text.

    - Converts tokens to numerical IDs using our word_to_idx.

    - Pads/truncates sequences to MAX_LEN.

    -Generates an attention_mask.

In [17]:
NUM_CPU_CORES = os.cpu_count()

In [18]:
# --- 3. Create a Preprocessing Function for dataset.map() ---
# This function will operate on a batch of examples because batched=True is recommended.
# It will use the globally defined nlp, CUSTOM_STOP_WORDS, word_to_idx, and MAX_LEN.
def preprocess_function(examples):
    # Initialize lists to store processed numerical inputs and masks for the batch
    input_ids_batch = []
    attention_mask_batch = []

    # Iterate through each text in the current batch
    for text in examples['text']:
        # 1. Tokenize and filter using your custom function
        processed_tokens = custom_tokenize_and_filter(text, nlp, CUSTOM_STOP_WORDS)
        
        # 2. Convert tokens to numerical IDs using the GLOBAL word_to_idx
        numerical_ids = [word_to_idx.get(token, word_to_idx[UNK_TOKEN]) for token in processed_tokens]
        
        # 3. Apply Padding/Truncation using the GLOBAL MAX_LEN
        if len(numerical_ids) < MAX_LEN:
            padded_ids = numerical_ids + [word_to_idx[PAD_TOKEN]] * (MAX_LEN - len(numerical_ids))
            mask = [1] * len(numerical_ids) + [0] * (MAX_LEN - len(numerical_ids))
        elif len(numerical_ids) > MAX_LEN:
            padded_ids = numerical_ids[:MAX_LEN]
            mask = [1] * MAX_LEN
        else: # Exactly MAX_LEN
            padded_ids = numerical_ids
            mask = [1] * MAX_LEN

        input_ids_batch.append(padded_ids)
        attention_mask_batch.append(mask)

    # Return a dictionary with the new columns. Hugging Face datasets will add these.
    # Ensure they are lists of lists or NumPy arrays for batched processing.
    return {
        'input_ids': input_ids_batch,
        'attention_mask': attention_mask_batch,
        # The original 'label' column is kept by default, or you could explicitly include it
        # 'labels': examples['label']
    }






# --- 4. Apply the Preprocessing Function to all splits using .map() ---
print("\nApplying preprocessing to dataset splits...")
# Remove original 'text' column if you don't need it anymore to save memory
processed_dataset = final_splits_reduced_dataset.map(
    preprocess_function,
    batched=True, # Process examples in batches for efficiency
    num_proc=NUM_CPU_CORES,
    remove_columns=['text'] # Remove original text column after processing
)



print(processed_dataset)
print(f"Example processed training item: {processed_dataset['train'][0]}")
print(f"Input IDs shape: {len(processed_dataset['train'][0]['input_ids'])}")
print(f"Attention Mask shape: {len(processed_dataset['train'][0]['attention_mask'])}")



# --- 5. Set the format for PyTorch DataLoaders ---
# This ensures that when you iterate, you get PyTorch tensors directly.
processed_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print("\nDataset format set to PyTorch:")
print(processed_dataset['train'][0])

# You can now create your PyTorch DataLoaders from these processed datasets:






Applying preprocessing to dataset splits...


Map (num_proc=4):   0%|          | 0/40000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})
Example processed training item: {'label': 1, 'input_ids': [15622, 18076, 3817, 7, 3104, 657, 1404, 4923, 16, 17, 2, 9121, 2908, 3, 496, 81, 574, 334, 9385, 243, 356, 1600, 26194, 2848, 32005, 10923, 526, 39, 1275, 22577, 40, 14516, 137, 72, 530, 5, 5, 901, 1188, 5, 465, 543, 34895, 1497, 31, 536, 126, 17501, 1188, 34895, 6, 32005, 24823, 40, 3, 150, 956, 78, 10924, 824, 6650, 39, 5512, 611, 3, 450, 2162, 48, 6, 3, 2403, 921, 9385, 653, 1123, 366, 2403, 2162, 1497, 2529, 370, 26195, 2162, 368, 26195, 75, 2162, 62, 396, 3, 25, 948, 26195, 128, 956, 88, 3, 33, 4, 25, 31, 32005, 5, 78, 34896, 12971, 412, 6784, 51501, 312, 4864, 51502, 445, 

In [22]:

save_directory = "./my_processed_dataset"
processed_dataset.save_to_disk(save_directory)

Saving the dataset (0/1 shards):   0%|          | 0/40000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [23]:
from IPython.display import FileLink

dataset_folder_path = "/kaggle/working/my_processed_dataset"

# Define the name for your zip file (this will be created in /kaggle/working/)
zip_file_name = "my_processed_dataset.zip"

# Ensure you are in the /kaggle/working/ directory before zipping
# This makes it easier for the zip command to find the folder.
os.chdir('/kaggle/working/')

# Use the 'zip' command to compress the folder
# -r: recursively include directories and their contents (including subfolders)
# The command will create 'my_processed_dataset.zip' containing the
# 'my_processed_dataset' folder and all its contents.
!zip -r {zip_file_name} {dataset_folder_path}

# Generate a downloadable link
FileLink(zip_file_name)

print(f"\nYour dataset folder has been zipped to '{zip_file_name}'. Click the link above to download it to your local machine.")

  adding: kaggle/working/my_processed_dataset/ (stored 0%)
  adding: kaggle/working/my_processed_dataset/dataset_dict.json (deflated 5%)
  adding: kaggle/working/my_processed_dataset/test/ (stored 0%)
  adding: kaggle/working/my_processed_dataset/test/dataset_info.json (deflated 70%)
  adding: kaggle/working/my_processed_dataset/test/data-00000-of-00001.arrow (deflated 96%)
  adding: kaggle/working/my_processed_dataset/test/state.json (deflated 39%)
  adding: kaggle/working/my_processed_dataset/train/ (stored 0%)
  adding: kaggle/working/my_processed_dataset/train/dataset_info.json (deflated 70%)
  adding: kaggle/working/my_processed_dataset/train/data-00000-of-00001.arrow (deflated 96%)
  adding: kaggle/working/my_processed_dataset/train/state.json (deflated 39%)
  adding: kaggle/working/my_processed_dataset/validation/ (stored 0%)
  adding: kaggle/working/my_processed_dataset/validation/dataset_info.json (deflated 70%)
  adding: kaggle/working/my_processed_dataset/validation/data-000

# Creating DataLoaders

In [24]:
## Important Hyperparameter
BATCH_SIZE = 32

def custom_collate_fn(batch):
    # 'batch' is a list of dictionaries, one for each sample in the batch
    # e.g., [{'input_ids': tensor, 'attention_mask': tensor, 'label': tensor}, ...]

    input_ids = torch.stack([item['input_ids'].to(torch.long) for item in batch])
    attention_mask = torch.stack([item['attention_mask'].to(torch.long) for item in batch])
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.float32)

    # You can return a tuple or a dictionary
    # Returning a tuple matches your original loop expectation (if you only want 2 items)
    return (input_ids, attention_mask, labels) # This would still be 3 items




### Creating DataLoader
train_dataloader = DataLoader(processed_dataset['train'], batch_size=BATCH_SIZE,collate_fn = custom_collate_fn, shuffle=True,num_workers = 4)
val_dataloader = DataLoader(processed_dataset['validation'],collate_fn = custom_collate_fn, batch_size=BATCH_SIZE,num_workers =4)
test_dataloader = DataLoader(processed_dataset['test'],collate_fn = custom_collate_fn, batch_size=BATCH_SIZE,num_workers = 4)


print(f"\nFirst batch from DataLoader (example):")
for batch_input_ids,batch_label,batch_attention_mask in train_dataloader:
    
    print(f"Input IDs batch shape: {type(batch_input_ids.shape)}")
    print(f"Attention Mask batch shape: {batch_attention_mask.shape}")
    print(f"Labels batch shape: {batch_label.shape}")
    break


First batch from DataLoader (example):
Input IDs batch shape: <class 'torch.Size'>
Attention Mask batch shape: torch.Size([32])
Labels batch shape: torch.Size([32, 1337])


## Loading Glove Embeddings and Creating the Embedded Matrix for the entire pipeline

In [31]:

GLOVE_PATH = '/kaggle/input/glove6b100dtxt/glove.6B.100d.txt' # Make sure this file is in the same directory or provide full path
GLOVE_EMBEDDING_DIM = 100 # Match the dimension of your downloaded GloVe file

glove_embeddings = {}
try:
    with open(GLOVE_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_embeddings[word] = vector
    print(f"Loaded {len(glove_embeddings)} words from GloVe.")
except FileNotFoundError:
    print(f"Error: GloVe file not found at {GLOVE_PATH}. Please download it from https://nlp.stanford.edu/projects/glove/.")
    print("For example, download glove.6B.zip and extract glove.6B.100d.txt.")
    exit() # Exit if GloVe not found, as it's critical for this part






Loaded 400000 words from GloVe.


In [32]:
# --- Step 3: Create the Embedding Matrix for Your Model ---
# Initialize with random values (or zeros) for words not found in GloVe
# Ensure the matrix has the correct dimensions: (your_vocab_size, embedding_dim)
embedding_matrix = np.random.uniform(-0.25, 0.25, (VOCAB_SIZE, GLOVE_EMBEDDING_DIM))

# Fill the matrix with GloVe vectors where available
for word, idx in word_to_idx.items():
    if word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[word]
    else:
        # Handle Out-Of-Vocabulary (OOV) words from your dataset that are not in GloVe
        # For UNK_TOKEN, it will be randomly initialized, which is fine.
        # For PAD_TOKEN, it remains randomly initialized, or you could explicitly set it to zeros.
        if word == PAD_TOKEN:
            embedding_matrix[idx] = np.zeros(GLOVE_EMBEDDING_DIM) # Set PAD to zeros
        # Other OOV words in your vocab will remain randomly initialized.

# Convert to a PyTorch tensor
embedding_matrix_tensor = torch.tensor(embedding_matrix, dtype=torch.float32)

print(f"\nShape of the final embedding matrix for your model: {embedding_matrix_tensor.shape}")


Shape of the final embedding matrix for your model: torch.Size([128465, 100])


In [33]:
# --- Step 4: Integrate into your VanillaRNN model ---

class VanillaRNNWithPretrainedEmbeddings(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim=100, output_dim=1, freeze_embeddings=True):
        super().__init__()
        
        # Create the embedding layer using nn.Embedding.from_pretrained
        # It automatically sets num_embeddings and embedding_dim from the matrix shape
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=freeze_embeddings)
        
        self.rnn = nn.RNN(self.embedding.embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_ids, attention_mask):
        embeddings = self.embedding(input_ids)
        
        lengths = attention_mask.sum(dim=1).cpu() # .cpu() might be needed for pack_padded_sequence older versions
        lengths = torch.max(lengths, torch.tensor(1, device=lengths.device))

        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )

        packed_hidden_states, final_hidden_state = self.rnn(packed_embeddings)

        final_state_squeezed = final_hidden_state.squeeze(0)
        output = self.fc(final_state_squeezed)

        return output

# --- Instantiate and Check Summary ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model with your prepared embedding matrix
# Set freeze_embeddings=True to keep them static (recommended for small datasets)
model_pretrained_emb = VanillaRNNWithPretrainedEmbeddings(
    embedding_matrix=embedding_matrix_tensor,
    freeze_embeddings=True # This is crucial!
).to(device)

print("\n--- Model Parameters with Pre-trained Embeddings ---")
total_params = sum(p.numel() for p in model_pretrained_emb.parameters())
trainable_params = sum(p.numel() for p in model_pretrained_emb.parameters() if p.requires_grad)
non_trainable_params = sum(p.numel() for p in model_pretrained_emb.parameters() if not p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {non_trainable_params}")

# Prepare dummy inputs for torchinfo summary
batch_size = 32 # Can be larger than 1 since embeddings are smaller
MAX_LEN = 50 # Example max sequence length
dummy_input_ids = torch.randint(0, VOCAB_SIZE, (batch_size, MAX_LEN), dtype=torch.long).to(device)
dummy_attention_mask = torch.ones((batch_size, MAX_LEN), dtype=torch.long).to(device)
for i in range(batch_size):
    rand_len = torch.randint(low=1, high=MAX_LEN + 1, size=(1,)).item()
    dummy_attention_mask[i, rand_len:] = 0

from torchinfo import summary
print("\n--- Model Summary (VanillaRNNWithPretrainedEmbeddings) ---")
summary(model_pretrained_emb, input_data=(dummy_input_ids, dummy_attention_mask), device=str(device))


--- Model Parameters with Pre-trained Embeddings ---
Total parameters: 12866801
Trainable parameters: 20301
Non-trainable parameters: 12846500

--- Model Summary (VanillaRNNWithPretrainedEmbeddings) ---


Layer (type:depth-idx)                   Output Shape              Param #
VanillaRNNWithPretrainedEmbeddings       [32, 1]                   --
├─Embedding: 1-1                         [32, 50, 100]             (12,846,500)
├─RNN: 1-2                               [751, 100]                20,200
├─Linear: 1-3                            [32, 1]                   101
Total params: 12,866,801
Trainable params: 20,301
Non-trainable params: 12,846,500
Total mult-adds (Units.GIGABYTES): 1.93
Input size (MB): 0.03
Forward/backward pass size (MB): 1.88
Params size (MB): 51.47
Estimated Total Size (MB): 53.37

In [34]:
HIDDEN_DIM = 128

# Attention Classes


In [35]:

## Assuming hidden_dim is the size of the hidden state of the RNN/encoder
class BahdanauAttention(nn.Module):
    def __init__(self, decoder_hidden_dim, encoder_hidden_dim,attention_hidden_dim):
        super(BahdanauAttention, self).__init__()
        # W_1 (also sometimes called W_query or W_decoder)
        self.W1 = nn.Linear(decoder_hidden_dim,attention_hidden_dim, bias=False) ## d_decoder * d_attention
        # W_2 (also sometimes called W_key or W_encoder)
        self.W2 = nn.Linear(encoder_hidden_dim, attention_hidden_dim, bias=False) ## d_encoder * d_attention
        # v_a (also sometimes called V or W_score)
        self.V = nn.Linear(attention_hidden_dim, 1, bias=False)   ## d_attention * 1     

    def forward(self, query_state, encoder_hidden_states, attention_mask=None):
        # query_state: (batch_size, hidden_dim) - this is typically the decoder's current hidden state
        # encoder_hidden_states: (batch_size, seq_len, hidden_dim) - these are the outputs from the encoder

        # Step 1 & 2: Transform the query and keys
        # transformed_query: (batch_size, hidden_dim)
        transformed_query = self.W1(query_state)
        # transformed_encoder_states: (batch_size, seq_len, hidden_dim)
        transformed_encoder_states = self.W2(encoder_hidden_states)

        # Step 3: Combine (Add) and Apply Non-Linearity
        # We need to unsqueeze transformed_query to (batch_size, 1, hidden_dim)
        # so it can be broadcasted and added to transformed_encoder_states.
        # energies: (batch_size, seq_len, hidden_dim)
        energies = torch.tanh(transformed_query.unsqueeze(1) + transformed_encoder_states)

        # Step 4: Calculate Raw Attention Scores (Energies)
        # attention_scores: (batch_size, seq_len, 1)
        attention_scores = self.V(energies).squeeze(2) # squeeze to (batch_size, seq_len)

        # Apply attention mask if provided (important for padded sequences!)
        if attention_mask is not None:
            # Mask out padded positions. Fill with a very small negative number
            # so that after softmax, they become virtually 0.
            attention_scores = attention_scores.masked_fill(attention_mask == 0, -1e9)

        # Step 5: Compute Attention Weights
        # attention_weights: (batch_size, seq_len) - sum to 1 across seq_len dim
        attention_weights = F.softmax(attention_scores, dim=1)

        # Step 6: Calculate Context Vector
       
        # context_vector: (batch_size, hidden_dim)
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_hidden_states).squeeze(1)
        
        return context_vector, attention_weights
    




class LuongDotProductAttention(nn.Module):
    def __init__(self,decoder_hidden_dim, encoder_hidden_dim):
        super(LuongDotProductAttention, self).__init__()

        if decoder_hidden_dim != encoder_hidden_dim:
            raise ValueError("Decoder hidden dimension must match encoder hidden dimension for Luong dot product attention.")
        # W_1 (also sometimes called W_query or W_decoder)

    def forward(self,query_state, encoder_hidden_states,attention_mask = None):

        # (B,D) -> (B,D,1)
        # (B,L,D) @ (B,D,1) -> (B,L,1)

        attention_scores = torch.bmm(encoder_hidden_states,query_state.unsqueeze(2)).squeeze(2)

        if attention_mask is not None:
            # Mask out padded positions. Fill with a very small negative number
            # so that after softmax, they become virtually 0.
            attention_scores = attention_scores.masked_fill(attention_mask == 0, -1e9)

        attention_weights = F.softmax(attention_scores, dim=1)


        #  (B,L) => (B,1,L) @ (B,L,D) => (B,1,D) =>(B,D)

        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_hidden_states).squeeze(1)
       

        return context_vector, attention_weights
    


class LuongGeneralAttention(nn.Module):
    def __init__(self,decoder_hidden_dim, encoder_hidden_dim):
        super(LuongGeneralAttention, self).__init__()

     
        # W_a
        self.W_a = nn.Linear(decoder_hidden_dim,encoder_hidden_dim ,bias=False)  ### d_h  -->d_e 
   


    def forward(self,query_state, encoder_hidden_states,attention_mask = None):

        # (B,D) -> (B,D,1)
        # (B,L,D) @ (B,D,1) -> (B,L,1)
        query_state_transformed = self.W_a(query_state)

        attention_scores = torch.bmm(encoder_hidden_states,query_state_transformed.unsqueeze(2)).squeeze(2)






        if attention_mask is not None:
            # Mask out padded positions. Fill with a very small negative number
            # so that after softmax, they become virtually 0.
            attention_scores = attention_scores.masked_fill(attention_mask == 0, -1e9)

        attention_weights = F.softmax(attention_scores, dim=1)


        #  (B,L) => (B,1,L) @ (B,L,D) => (B,1,D) =>(B,D)

        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_hidden_states).squeeze(1)
       

        return context_vector, attention_weights
    


class LuongConcatAttention(nn.Module):
    def __init__(self,decoder_hidden_dim, encoder_hidden_dim,attention_hidden_dim):
        super(LuongConcatAttention, self).__init__()

     
        # W_c
        self.W_c = nn.Linear(decoder_hidden_dim + encoder_hidden_dim,attention_hidden_dim, bias=False)

        self.V = nn.Linear(attention_hidden_dim, 1, bias=False)   ## d_attention * 1    


    def forward(self,query_state, encoder_hidden_states,attention_mask = None):

        # (B,D_d) => (B,1,D_d) => (B,L,D_d)
        transformed_query = query_state.unsqueeze(1).expand(-1, encoder_hidden_states.size(1), -1)  # (B, L, D_d)





        concat_query_encoder_hidden_states = torch.cat((transformed_query,encoder_hidden_states),dim=2)  #(B, L, D_d + D_e)



        energies = torch.tanh(self.W_c(concat_query_encoder_hidden_states))#(B,L,D_a)


    
        # attention_scores: (batch_size, seq_len, 1)
        attention_scores = self.V(energies).squeeze(2) # squeeze to (batch_size, seq_len)

        if attention_mask is not None:
            # Mask out padded positions. Fill with a very small negative number
            # so that after softmax, they become virtually 0.
            attention_scores = attention_scores.masked_fill(attention_mask == 0, -1e9)

        attention_weights = F.softmax(attention_scores, dim=1)


        #  (B,L) => (B,1,L) @ (B,L,D) => (B,1,D) =>(B,D)

        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_hidden_states).squeeze(1)
       

        return context_vector, attention_weights

# Vanilla Models

In [36]:
class VanillaRNN(nn.Module):
    def __init__(self,embedding_matrix, hidden_dim=HIDDEN_DIM, output_dim=1, freeze_embeddings=True):
        super().__init__()
         # Create the embedding layer using nn.Embedding.from_pretrained
        # It automatically sets num_embeddings and embedding_dim from the matrix shape
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=freeze_embeddings)
        self.rnn = nn.RNN(self.embedding.embedding_dim, hidden_dim, batch_first=True) # batch_first=True is important
        self.fc = nn.Linear(hidden_dim, output_dim) # Output dimension for binary classification

    def forward(self, input_ids, attention_mask):
        # 1. Embed the input_ids
        # input_ids shape: (batch_size, sequence_length)
        # embeddings shape: (batch_size, sequence_length, embedding_dim)
        embeddings = self.embedding(input_ids)

        # 2. Get original (non-padded) sequence lengths from attention_mask
        # For each sequence in the batch, sum the '1's in its attention_mask.
        # Ensure lengths are on CPU for pack_padded_sequence if your data is on GPU.
        # This will be a 1D tensor of shape (batch_size,)
        lengths = attention_mask.sum(dim=1).cpu()

        # Handle potential zero-length sequences (though unlikely in sentiment analysis)
        # pack_padded_sequence requires lengths to be > 0.
        # If any length is zero, set it to 1 to avoid error, and mask its contribution later if needed.
        # For typical tokenization, min length will be 2 ([CLS], [SEP]).
        # If there's a risk of 0 lengths, you might replace them with 1 and then zero out their contribution to loss.
        # For most practical cases with a tokenizer adding special tokens, this isn't strictly necessary
        # but good to be aware of.
        lengths = torch.max(lengths, torch.tensor(1, device=lengths.device))

        # 3. Pack the padded sequence
        # This prevents the RNN from processing padding tokens.
        # It creates a PackedSequence object that the RNN understands.
        # embeddings shape: (batch_size, sequence_length, embedding_dim)
        # lengths shape: (batch_size,)
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False # enforce_sorted=False if your lengths aren't sorted
        )

        # 4. Pass the packed sequence through the RNN
        # hidden_states: PackedSequence object containing output for non-padded tokens
        # final_hidden_state: Tensor containing the last hidden state for each sequence (h_n, c_n for LSTM)
        # Note: for RNN, final_hidden_state is (num_layers * num_directions, batch_size, hidden_size)
        # For a simple, non-bidirectional RNN with 1 layer, this is (1, batch_size, hidden_size)
        _, final_hidden_state = self.rnn(packed_embeddings)

        # 5. Unpack the hidden states (optional, but useful if you plan to use attention on all hidden states)
        # This converts the PackedSequence back to a padded tensor with original batch_first structure.
        # padded_hidden_states, _ = nn.utils.rnn.pad_packed_sequence(
        #     packed_hidden_states, batch_first=True, total_length=input_ids.shape[1]
        # )
        # padded_hidden_states shape: (batch_size, sequence_length, hidden_dim)

        # 6. Use the final hidden state for classification
        # For a simple RNN, the final_hidden_state (h_n) represents the context of the sequence.
        # Since batch_first=True in the RNN, final_hidden_state is (1, batch_size, hidden_dim).
        # We need to remove the first dimension (num_layers * num_directions) to get (batch_size, hidden_dim).
        # If it were bidirectional, final_hidden_state would be (2, batch_size, hidden_dim),
        # requiring concatenation of forward/backward states before the fc layer.
        final_state_squeezed = final_hidden_state.squeeze(0) # shape: (batch_size, hidden_dim)

        # 7. Pass through the final linear layer
        # output will be logits, shape: (batch_size, output_dim)
        output = self.fc(final_state_squeezed)

        return output



class VanillaLSTM(nn.Module):
    def __init__(self,embedding_matrix, hidden_dim=HIDDEN_DIM, output_dim=1,freeze_embeddings=True):
        super().__init__()
        self.embedding =  nn.Embedding.from_pretrained(embedding_matrix, freeze=freeze_embeddings)
        self.lstm = nn.LSTM(self.embedding.embedding_dim, hidden_dim, batch_first=True) # batch_first=True is important
        self.fc = nn.Linear(hidden_dim, output_dim) # Output dimension for binary classification

    def forward(self, input_ids, attention_mask):
        # 1. Embed the input_ids
        embeddings = self.embedding(input_ids)

        # 2. Get original (non-padded) sequence lengths from attention_mask
        lengths = attention_mask.sum(dim=1).cpu()

        # 3. Pack the padded sequence
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )

        # 4. Pass the packed sequence through the LSTM
        _, (final_hidden_state, _) = self.lstm(packed_embeddings)

        # 5. Use the final hidden state for classification
        final_state_squeezed = final_hidden_state.squeeze(0) # shape: (batch_size, hidden_dim)

        # 6. Pass through the final linear layer
        output = self.fc(final_state_squeezed)

        return output
    
class VanillaBidirectionalRNN(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim=HIDDEN_DIM, output_dim=1,freeze_embeddings=True):
        super().__init__()
        self.embedding =  nn.Embedding.from_pretrained(embedding_matrix, freeze=freeze_embeddings)
        self.rnn = nn.RNN(self.embedding.embedding_dim, hidden_dim, batch_first=True, bidirectional=True) # bidirectional=True
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # Output dimension for binary classification

    def forward(self, input_ids, attention_mask):
        # 1. Embed the input_ids
        embeddings = self.embedding(input_ids)

        # 2. Get original (non-padded) sequence lengths from attention_mask
        lengths = attention_mask.sum(dim=1).cpu()

        # 3. Pack the padded sequence
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )

        # 4. Pass the packed sequence through the RNN
        _, final_hidden_state = self.rnn(packed_embeddings)

        # 5. Use the final hidden state for classification
        # For bidirectional RNNs, final_hidden_state will have shape (2, batch_size, hidden_dim)
        final_state_squeezed = final_hidden_state.view(final_hidden_state.size(1), -1) # shape: (batch_size, hidden_dim * 2)

        # 6. Pass through the final linear layer
        output = self.fc(final_state_squeezed)  




        return output
    


class VanillaBidirectionalLSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim=HIDDEN_DIM, output_dim=1, freeze_embeddings=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze_embeddings)  # Freeze embeddings by default
        self.lstm = nn.LSTM(self.embedding.embedding_dim, hidden_dim, batch_first=True, bidirectional=True) # bidirectional=True
        self.fc = nn.Linear(hidden_dim * 2, output_dim) # Output dimension for binary classification

    def forward(self, input_ids, attention_mask):
        # 1. Embed the input_ids
        embeddings = self.embedding(input_ids)

        # 2. Get original (non-padded) sequence lengths from attention_mask
        lengths = attention_mask.sum(dim=1).cpu()

        # 3. Pack the padded sequence
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )

        # 4. Pass the packed sequence through the LSTM
        _, (final_hidden_state, _) = self.lstm(packed_embeddings)

        # 5. Use the final hidden state for classification
        final_state_squeezed = final_hidden_state.view(final_hidden_state.size(1), -1) # shape: (batch_size, hidden_dim * 2)

        # 6. Pass through the final linear layer
        output = self.fc(final_state_squeezed)                              

        return output







# General Purpose Models with Plug and Play Attention

In [37]:


# --- General Purpose RNN Model ---
class VanillaRNNWithPlugAndPlayAttention(nn.Module):
    def __init__(self,embedding_matrix, attention_class, attention_kwargs,return_attention_weights =False,
                  hidden_dim=HIDDEN_DIM, output_dim=1,freeze_embeddings=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix,freeze= freeze_embeddings)
           # We need to ensure the RNN outputs all hidden states. batch_first=True is important.
        self.rnn = nn.RNN(self.embedding.embedding_dim, hidden_dim, batch_first=True)
        
        # Instantiate the attention module using the provided class and kwargs
        # The **attention_kwargs unpacks the dictionary into named arguments
        self.attention = attention_class(
            decoder_hidden_dim=hidden_dim, # Decoder's hidden state is the query for attention
            encoder_hidden_dim=hidden_dim, # Encoder's outputs are the keys/values for attention
            **attention_kwargs # This will pass specific kwargs like 'attention_dim' if present
        )
        self.return_attention_weights = return_attention_weights
  


        # The final fully connected layer will now take the concatenated 
        # final hidden state AND the context vector from attention.
        # So, its input dimension becomes hidden_dim + hidden_dim = 2 * hidden_dim

        # Assuming context_vector will also be of hidden_dim size (typical)   (! I don't find where it isn't the case!)

        self.fc = nn.Linear(hidden_dim * 2, output_dim) 

    def forward(self, input_ids, attention_mask):
                # 1. Embed the input_ids
        embeddings = self.embedding(input_ids)

                # 2. Get original (non-padded) sequence lengths from attention_mask
        lengths = attention_mask.sum(dim=1).cpu()
   # 3. Pack the padded sequence
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )
     
        # 4. Pass the packed sequence through the RNN
        packed_output, final_hidden_state = self.rnn(packed_embeddings)

      # 5. Unpack the RNN outputs to get all encoder_hidden_states

        outputs, _ = nn.utils.rnn.pad_packed_sequence(
            packed_output, batch_first=True, total_length=input_ids.size(1)
        )
          # 6 Squeeze the final_hidden_state to (batch_size, hidden_dim) to use as query
        query = final_hidden_state.squeeze(0) 

        # Calculate context vector using the plug-and-play attention module
        context_vector, attention_weights = self.attention(
            query_state=query, 
            encoder_hidden_states=outputs, 
            attention_mask=attention_mask
        )
 # 7. Concatenate the query (final hidden state) with the context vector
        # This combines the "sequential summary" with the "attention-weighted summary"
        combined_representation = torch.cat((query, context_vector), dim=1)
        output = self.fc(combined_representation)  

        if self.return_attention_weights:
            # If we want to return attention weights for visualization/debugging
            return output, attention_weights
        else:
            return output 




class VanillaBidirectionalRNNWithPlugAndPlayAttention(nn.Module):
    def __init__(self,embedding_matrix, attention_class, attention_kwargs,return_attention_weights =False,
                 hidden_dim=HIDDEN_DIM, output_dim=1,freeze_embeddings=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix,freeze= freeze_embeddings)
        self.rnn = nn.RNN(self.embedding.embedding_dim, hidden_dim, batch_first=True, bidirectional=True) # bidirectional=True
        self.fc = nn.Linear(hidden_dim * 4, output_dim) # Output dimension for binary classification


        self.attention = attention_class(
            decoder_hidden_dim=2*hidden_dim, # Decoder's hidden state is the query for attention
            encoder_hidden_dim=2*hidden_dim, # Encoder's outputs are the keys/values for attention
            **attention_kwargs # This will pass specific kwargs like 'attention_dim' if present
        )
        self.return_attention_weights = return_attention_weights


    def forward(self, input_ids, attention_mask):
        # 1. Embed the input_ids
        embeddings = self.embedding(input_ids) #(B,L,E)

        # 2. Get original (non-padded) sequence lengths from attention_mask
        lengths = attention_mask.sum(dim=1).cpu()

        # 3. Pack the padded sequence
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )

        # 4. Pass the packed sequence through the RNN
        # packed_hidden_states: PackedSequence of (B, L, 2*H)
        # final_hidden_state: (num_layers * num_directions, B, H) -> (2, B, H) for 1-layer bidirectional
        packed_hidden_states, final_hidden_state = self.rnn(packed_embeddings)
            
        # 5. Unpack the rnn outputs to get all encoder_hidden_states
        # outputs: (B, L, 2*H) - this contains the concatenated forward and backward hidden states for each time step

        outputs, _ = nn.utils.rnn.pad_packed_sequence(
            packed_hidden_states, batch_first=True, total_length=input_ids.size(1)
        )
        # For bidirectional RNNs, final_hidden_state will have shape (2, batch_size, hidden_dim)
        #6 Squeeze the final_hidden_state  (2,batch_size,hidden_dim) to (batch_size, 2*hidden_dim) to use as query
        query = final_hidden_state.view(final_hidden_state.size(1), -1) # shape: (batch_size, hidden_dim * 2)
        #7Calculate context vector using the plug-and-play attention module
        context_vector, attention_weights = self.attention(
            query_state=query, 
            encoder_hidden_states=outputs, 
            attention_mask=attention_mask
        )

    
       


        # 8. Concatenate the query (final hidden state) with the context vector
        # query: (B, 2*H)
        # context_vector: (B, 2*H) - typically, attention outputs context vector of encoder_hidden_dim
        # combined_representation: (B, 4*H)
        # This combines the "sequential summary" with the "attention-weighted summary"


        combined_representation = torch.cat((query, context_vector), dim=1)    #(B,2*H)  +(B,2*hidden_dim) = (B,4*hidden_dim)
        

      
        # 9. Pass through the final linear layer
        output = self.fc(combined_representation)  



 # Return attention weights for visualization/debugging
  
        if self.return_attention_weights:
        
            return output, attention_weights
        else:
            return output  # Only return the output if weights are not explicitly requested



# --- General Purpose LSTM Model ---
class VanillaLSTMWithPlugAndPlayAttention(nn.Module):
    def __init__(self,embedding_matrix, attention_class, attention_kwargs,return_attention_weights =False,
                 hidden_dim=HIDDEN_DIM, output_dim=1,freeze_embeddings=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix,freeze= freeze_embeddings)
           # We need to ensure the RNN outputs all hidden states. batch_first=True is important.
        self.lstm = nn.LSTM(self.embedding.embedding_dim, hidden_dim, batch_first=True)
        
        # Instantiate the attention module using the provided class and kwargs
        # The **attention_kwargs unpacks the dictionary into named arguments
        self.attention = attention_class(
            decoder_hidden_dim=hidden_dim, # Decoder's hidden state is the query for attention
            encoder_hidden_dim=hidden_dim, # Encoder's outputs are the keys/values for attention
            **attention_kwargs # This will pass specific kwargs like 'attention_dim' if present
        )
        self.return_attention_weights = return_attention_weights
  


        # The final fully connected layer will now take the concatenated 
        # final hidden state AND the context vector from attention.
        # So, its input dimension becomes hidden_dim + hidden_dim = 2 * hidden_dim

        # Assuming context_vector will also be of hidden_dim size (typical)   (! I don't find where it isn't the case!)

        self.fc = nn.Linear(hidden_dim * 2, output_dim) 

    def forward(self, input_ids, attention_mask):
                # 1. Embed the input_ids
        embeddings = self.embedding(input_ids)

                # 2. Get original (non-padded) sequence lengths from attention_mask
        lengths = attention_mask.sum(dim=1).cpu()
   # 3. Pack the padded sequence
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )
     
        # 4. Pass the packed sequence through the lstm
        packed_output, (final_hidden_state, _)  = self.lstm(packed_embeddings)

      # 5. Unpack the LSTM outputs to get all encoder_hidden_states

        outputs, _ = nn.utils.rnn.pad_packed_sequence(
            packed_output, batch_first=True, total_length=input_ids.size(1)
        )
          # 6 Squeeze the final_hidden_state to (batch_size, hidden_dim) to use as query
        query = final_hidden_state.squeeze(0) 

        # Calculate context vector using the plug-and-play attention module
        context_vector, attention_weights = self.attention(
            query_state=query, 
            encoder_hidden_states=outputs, 
            attention_mask=attention_mask
        )
 # 7. Concatenate the query (final hidden state) with the context vector
        # This combines the "sequential summary" with the "attention-weighted summary"
        combined_representation = torch.cat((query, context_vector), dim=1)
        output = self.fc(combined_representation)  


        # Conditionally return attention weights on basis of flags

        # Allows for visualization or debugging of attention mechanisms
        if self.return_attention_weights:
            return output, attention_weights
        else:
            return output 



## General Purpose Bidirectional LSTM Model
class VanillaBidirectionalLSTMWithPlugAndPlayAttention(nn.Module):
    def __init__(self,embedding_matrix, attention_class, attention_kwargs,return_attention_weights =False,
                 hidden_dim=HIDDEN_DIM, output_dim=1,freeze_embeddings=True):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix,freeze= freeze_embeddings)
        self.lstm = nn.LSTM(self.embedding.embedding_dim, hidden_dim, batch_first=True, bidirectional=True) # bidirectional=True
        self.fc = nn.Linear(hidden_dim * 4, output_dim) # Output dimension for binary classification


        self.attention = attention_class(
            decoder_hidden_dim=2*hidden_dim, # Decoder's hidden state is the query for attention
            encoder_hidden_dim=2*hidden_dim, # Encoder's outputs are the keys/values for attention
            **attention_kwargs # This will pass specific kwargs like 'attention_dim' if present
        )
        self.return_attention_weights = return_attention_weights


    def forward(self, input_ids, attention_mask):
        # 1. Embed the input_ids
        embeddings = self.embedding(input_ids) #(B,L,E)

        # 2. Get original (non-padded) sequence lengths from attention_mask
        lengths = attention_mask.sum(dim=1).cpu()

        # 3. Pack the padded sequence
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(
            embeddings, lengths, batch_first=True, enforce_sorted=False
        )

        # 4. Pass the packed sequence through the LSTM
        # packed_hidden_states: PackedSequence of (B, L, 2*H)
        # final_hidden_state: (num_layers * num_directions, B, H) -> (2, B, H) for 1-layer bidirectional
        packed_hidden_states, (final_hidden_state, _) = self.lstm(packed_embeddings)
            
        # 5. Unpack the rnn outputs to get all encoder_hidden_states
        # outputs: (B, L, 2*H) - this contains the concatenated forward and backward hidden states for each time step

        outputs, _ = nn.utils.rnn.pad_packed_sequence(
            packed_hidden_states, batch_first=True, total_length=input_ids.size(1)
        )
        # For bidirectional LSTMs, final_hidden_state will have shape (2, batch_size, hidden_dim)
        #6 Squeeze the final_hidden_state  (2,batch_size,hidden_dim) to (batch_size, 2*hidden_dim) to use as query
        query = final_hidden_state.view(final_hidden_state.size(1), -1) # shape: (batch_size, hidden_dim * 2)
        #7Calculate context vector using the plug-and-play attention module
        context_vector, attention_weights = self.attention(
            query_state=query, 
            encoder_hidden_states=outputs, 
            attention_mask=attention_mask
        )

    
       


        # 8. Concatenate the query (final hidden state) with the context vector
        # query: (B, 2*H)
        # context_vector: (B, 2*H) - typically, attention outputs context vector of encoder_hidden_dim
        # combined_representation: (B, 4*H)
        # This combines the "sequential summary" with the "attention-weighted summary"


        combined_representation = torch.cat((query, context_vector), dim=1)    #(B,2*H)  +(B,2*hidden_dim) = (B,4*hidden_dim)
        

      
        # 9. Pass through the final linear layer
        output = self.fc(combined_representation)  

 # Return attention weights for visualization/debugging
  
        if self.return_attention_weights:
        
            return output, attention_weights
        else:
            return output  # Only return the output if weights are not explicitly requested


In [38]:

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Sneak through into Model Architectures

We will look into model architectures to judge the complexity of model by looking through the number of parameters

In [39]:
from torchinfo import summary

VanillaModels = [VanillaRNN,VanillaLSTM,VanillaBidirectionalRNN,VanillaBidirectionalLSTM]


for model_class in VanillaModels:
    model = model_class(embedding_matrix = embedding_matrix_tensor).to(device)


    dummy_input_ids = torch.randint(0, VOCAB_SIZE, (BATCH_SIZE, MAX_LEN), dtype=torch.long).to(device)
    dummy_attention_mask = torch.ones((batch_size, MAX_LEN), dtype=torch.long).to(device)
    for i in range(BATCH_SIZE):
        rand_len = torch.randint(low=1, high=MAX_LEN + 1, size=(1,)).item()
        dummy_attention_mask[i, rand_len:] = 0

    print(f"\n--- Model  Summary  {model_class.__name__} ---")
    print(summary(model ,input_data=(dummy_input_ids, dummy_attention_mask), device=str(device)))


--- Model  Summary  VanillaRNN ---
Layer (type:depth-idx)                   Output Shape              Param #
VanillaRNN                               [32, 1]                   --
├─Embedding: 1-1                         [32, 50, 100]             (12,846,500)
├─RNN: 1-2                               [757, 128]                29,440
├─Linear: 1-3                            [32, 1]                   129
Total params: 12,876,069
Trainable params: 29,569
Non-trainable params: 12,846,500
Total mult-adds (Units.GIGABYTES): 3.26
Input size (MB): 0.03
Forward/backward pass size (MB): 2.06
Params size (MB): 51.50
Estimated Total Size (MB): 53.59

--- Model  Summary  VanillaLSTM ---
Layer (type:depth-idx)                   Output Shape              Param #
VanillaLSTM                              [32, 1]                   --
├─Embedding: 1-1                         [32, 50, 100]             (12,846,500)
├─LSTM: 1-2                              [762, 128]                117,760
├─Linear: 1-3    

In [40]:

HIDDEN_DIM =128
attention_types = [BahdanauAttention,LuongDotProductAttention,LuongGeneralAttention,LuongConcatAttention]
attention_kwargs= [{"attention_hidden_dim":HIDDEN_DIM},{},{},{"attention_hidden_dim":HIDDEN_DIM}]
plug_and_play_attention_models = [VanillaRNNWithPlugAndPlayAttention,VanillaBidirectionalRNNWithPlugAndPlayAttention,VanillaLSTMWithPlugAndPlayAttention,VanillaBidirectionalLSTMWithPlugAndPlayAttention]

for pap_a_model in plug_and_play_attention_models:
    for attention_type,attention_kwarg in zip(attention_types,attention_kwargs):
        
        model = pap_a_model( embedding_matrix_tensor,attention_type,attention_kwarg).to(device)


        dummy_input_ids = torch.randint(0, VOCAB_SIZE, (BATCH_SIZE, MAX_LEN), dtype=torch.long).to(device)
        dummy_attention_mask = torch.ones((batch_size, MAX_LEN), dtype=torch.long).to(device)
        for i in range(BATCH_SIZE):
            rand_len = torch.randint(low=1, high=MAX_LEN + 1, size=(1,)).item()
            dummy_attention_mask[i, rand_len:] = 0

        print(f"\n--- Model  Summary  {pap_a_model.__name__} ---with {attention_type.__name__}")
        print(summary(model ,input_data=(dummy_input_ids, dummy_attention_mask), device=str(device)))


--- Model  Summary  VanillaRNNWithPlugAndPlayAttention ---with BahdanauAttention
Layer (type:depth-idx)                   Output Shape              Param #
VanillaRNNWithPlugAndPlayAttention       [32, 1]                   --
├─Embedding: 1-1                         [32, 50, 100]             (12,846,500)
├─RNN: 1-2                               [747, 128]                29,440
├─BahdanauAttention: 1-3                 [32, 128]                 --
│    └─Linear: 2-1                       [32, 128]                 16,384
│    └─Linear: 2-2                       [32, 50, 128]             16,384
│    └─Linear: 2-3                       [32, 50, 1]               128
├─Linear: 1-4                            [32, 1]                   257
Total params: 12,909,093
Trainable params: 62,593
Non-trainable params: 12,846,500
Total mult-adds (Units.GIGABYTES): 3.23
Input size (MB): 0.03
Forward/backward pass size (MB): 3.73
Params size (MB): 51.64
Estimated Total Size (MB): 55.39

--- Model  Summary

In [41]:
for attention,attention_kwarg in zip(attention_types,attention_kwargs):
    print(attention, "   ", attention_kwarg)


<class '__main__.BahdanauAttention'>     {'attention_hidden_dim': 128}
<class '__main__.LuongDotProductAttention'>     {}
<class '__main__.LuongGeneralAttention'>     {}
<class '__main__.LuongConcatAttention'>     {'attention_hidden_dim': 128}


# After this the traning procedure begins !

## important parameters 

In [42]:



learning_rate =  0.01
epochs = 30


# Training Pipeline

In [43]:

def train_model(model, learning_rate, epochs, train_dataloader, val_dataloader, model_save_dir=None, patience=5):
    """
    Trains a PyTorch model with early stopping, saves the best model,
    and logs/plots training and validation loss.

    Args:
        model (torch.nn.Module): The PyTorch model to train.
        learning_rate (float): The learning rate for the optimizer.
        epochs (int): The maximum number of training epochs.
        train_dataloader (torch.utils.data.DataLoader): DataLoader for training data.
        val_dataloader (torch.utils.data.DataLoader): DataLoader for validation data.
        model_save_dir (str, optional): The directory to save the model, logs, and plots.
                                        Example: "/kaggle/working/my_model_folder"
                                        If None, nothing will be saved.
        patience (int): Number of epochs to wait for improvement before stopping.
    """
    model.train()
    model = model.to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Variables for early stopping
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    best_epoch = -1 # To track which epoch had the best validation loss

    # Lists to store metrics for plotting and logging
    train_losses = []
    val_losses = []

    # Create the model directory if it's provided
    if model_save_dir:
        os.makedirs(model_save_dir, exist_ok=True)
        print(f"Model saving directory created/ensured: {model_save_dir}")
    start_time = time.time() # Record start time
    for epoch in range(epochs):
        # --- Training Phase ---
        model.train() # Set model to training mode
        current_train_loss = 0.0
        for batch_idx, (batch_input_ids, batch_attention_mask, batch_labels) in enumerate(train_dataloader):
            # Move data to the correct device
            batch_input_ids = batch_input_ids.to(device)
            batch_labels = batch_labels.to(device)
            batch_attention_mask = batch_attention_mask.to(device)

            # Forward pass
            output = model.forward(batch_input_ids, batch_attention_mask)
            loss = criterion(output, batch_labels.view(-1, 1).float())

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            current_train_loss += loss.item()

            # Optional: print per-batch loss if needed, but it can be very verbose
            # print(f"Epoch {epoch+1} | Batch {batch_idx+1}/{len(train_dataloader)} | Train Batch Loss: {loss.item():.4f}")

        avg_train_loss = current_train_loss / len(train_dataloader)
        train_losses.append(avg_train_loss)
        print(f'Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f}')

        # --- Validation Phase ---
        model.eval() # Set model to evaluation mode
        current_val_loss = 0.0
        with torch.no_grad(): # Disable gradient calculations for validation
            for batch_idx, (batch_input_ids, batch_attention_mask, batch_labels) in enumerate(val_dataloader):
                batch_input_ids = batch_input_ids.to(device)
                batch_labels = batch_labels.to(device)
                batch_attention_mask = batch_attention_mask.to(device)

                output = model.forward(batch_input_ids, batch_attention_mask)
                loss = criterion(output, batch_labels.view(-1, 1).float())
                current_val_loss += loss.item()

        avg_val_loss = current_val_loss / len(val_dataloader)
        val_losses.append(avg_val_loss)
        print(f'Epoch {epoch+1}/{epochs} | Val Loss: {avg_val_loss:.4f}')

        # --- Early Stopping Logic ---
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            best_epoch = epoch + 1
            # Save the best model state dictionary
            best_model_state = model.state_dict()
            print(f"Validation loss improved. Current best: {best_val_loss:.4f} (Epoch {best_epoch}).")
        else:
            epochs_no_improve += 1
            print(f"Validation loss did not improve for {epochs_no_improve} epoch(s). Best was {best_val_loss:.4f} at Epoch {best_epoch}.")
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs (patience: {patience}).")
                break # Exit the training loop

    # --- Post-training actions: Saving and Plotting ---
    end_time = time.time() # Record end time
    training_time_seconds = end_time - start_time
    print(f"Finished training for {model.__class__.__name__}. Total time: {training_time_seconds:.2f} seconds.")

    
    # Save the best model after training (either all epochs or early stopped)
    if model_save_dir and best_model_state is not None:
        model_path = os.path.join(model_save_dir, "best_model.pth")
        torch.save(best_model_state, model_path)
        print(f"Best model (based on validation loss) saved successfully to: {model_path}")
    elif model_save_dir is None:
        print("Model not saved (no model_save_dir provided).")
    else:
        print("Model not saved as validation loss never improved or no best state was recorded.")

    # Save training logs
    if model_save_dir:
        log_data = pd.DataFrame({
            'Epoch': range(1, len(train_losses) + 1),
            'Train_Loss': train_losses,
            'Val_Loss': val_losses
        })
        log_path = os.path.join(model_save_dir, "training_log.csv")
        log_data.to_csv(log_path, index=False)
        print(f"Training logs saved to: {log_path}")

        # Plotting the training and validation loss
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss', marker='o', linestyle='-')
        plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss', marker='x', linestyle='--')

        # Add a vertical line for the best epoch if early stopping occurred
        if best_epoch != -1 and best_epoch <= len(train_losses): # Ensure best_epoch is within plotted range
             plt.axvline(x=best_epoch, color='r', linestyle=':', label=f'Best Val Loss (Epoch {best_epoch})')


        plt.title('Training and Validation Loss Over Epochs', fontsize=16)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel('Loss', fontsize=12)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.xticks(range(1, len(train_losses) + 1)) # Ensure integer ticks for epochs
        plt.tight_layout() # Adjust layout to prevent labels from overlapping
        plot_path = os.path.join(model_save_dir, "loss_plot.png")
        plt.savefig(plot_path)
        print(f"Loss plot saved to: {plot_path}")
        plt.close() # Close the plot to free memory

    # Load the best model state if it was saved, before returning
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        model.to(device) # Ensure the loaded model is on the correct device
        print("Loaded best model state for return.")
    else:
        print("No best model state recorded or loaded.")

    return model, training_time_seconds

# Getting and Intepreting Results Pipeline

In [44]:
def get_statistics(model, test_dataloader, threshold=0.5):
    """
    Evaluates the model on the test_dataloader and returns various classification metrics.

    Args:
        model (torch.nn.Module): The trained PyTorch model.
        test_dataloader (torch.utils.data.DataLoader): DataLoader for the test set.
        threshold (float): The threshold for converting sigmoid output to binary predictions.
                           Defaults to 0.5.

    Returns:
        dict: A dictionary containing:
              - 'accuracy': Overall accuracy.
              - 'precision': Precision score.
              - 'recall': Recall score.
              - 'f1_score': F1-score.
              - 'confusion_matrix': A 2x2 NumPy array representing the confusion matrix.
    """
    model.eval()  # Set the model to evaluation mode

    # Initialize TorchMetrics for binary classification
    # Ensure task="binary" and threshold are set, and move metrics to the device
    metric_accuracy = Accuracy(task="binary", threshold=threshold).to(device)
    metric_precision = Precision(task="binary", threshold=threshold).to(device)
    metric_recall = Recall(task="binary", threshold=threshold).to(device)
    metric_f1 = F1Score(task="binary", threshold=threshold).to(device)
    metric_confusion_matrix = BinaryConfusionMatrix(threshold=threshold).to(device)

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch_input_ids, batch_attention_mask, batch_labels in test_dataloader:
            # Move data to the correct device
            batch_input_ids = batch_input_ids.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_labels = batch_labels.to(device)

            # Forward pass: model should accept both input_ids and attention_mask
            output_logits = model(batch_input_ids,batch_attention_mask)
            
            # Apply sigmoid to get probabilities. Squeeze to remove the last dimension (e.g., from [batch_size, 1] to [batch_size])
            output_probs = F.sigmoid(output_logits).squeeze(1)

            # Update each metric with the predicted probabilities and ground truth labels
            # Note: TorchMetrics expects target labels to be of type torch.long
            metric_accuracy.update(output_probs, batch_labels.long())
            metric_precision.update(output_probs, batch_labels.long())
            metric_recall.update(output_probs, batch_labels.long())
            metric_f1.update(output_probs, batch_labels.long())
            metric_confusion_matrix.update(output_probs, batch_labels.long())

    # Compute final metrics from the accumulated states
    accuracy = metric_accuracy.compute().item()
    precision = metric_precision.compute().item()
    recall = metric_recall.compute().item()
    f1_score = metric_f1.compute().item()
    
    # Confusion matrix is a tensor, move to CPU and convert to NumPy for printing/return
    confusion_matrix_tensor = metric_confusion_matrix.compute()
    confusion_matrix_np = confusion_matrix_tensor.cpu().numpy()

    # Print the results
    print("\n--- Classification Statistics ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1_score:.4f}")

    print("\nConfusion Matrix:")
    # For binary classification, a 2x2 matrix is typically structured as:
    # [[True Negatives, False Positives],
    #  [False Negatives, True Positives]]
    cm_df = pd.DataFrame(confusion_matrix_np,
                         index=['Actual Negative (0)', 'Actual Positive (1)'],
                         columns=['Predicted Negative (0)', 'Predicted Positive (1)'])
    print(cm_df)

    # Return the results as a dictionary
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'confusion_matrix': confusion_matrix_np
    }

# Final Traning of All Models!

In [45]:

# Define where to save models and log results
model_save_base_dir = "/kaggle/working/saved_models"
os.makedirs(model_save_base_dir, exist_ok=True) # Ensure base directory exists

results_log_file_path = "/kaggle/working/model_results.csv"



# --- Global header check for CSV logging ---
write_header = not os.path.exists(results_log_file_path)
if write_header:
    with open(results_log_file_path, 'w') as f_log: # 'w' to create/overwrite if it doesn't exist
        header = "Model_Name,Accuracy,Precision,Recall,F1_Score,CM_TN,CM_FP,CM_FN,CM_TP,Training_Time_Sec,Trainable_Params\n"
        f_log.write(header)
    print(f"Created new results log file: {results_log_file_path} with header.")
else:
    print(f"Appending to existing results log file: {results_log_file_path}.")




Created new results log file: /kaggle/working/model_results.csv with header.


In [46]:
def count_parameters(model):
    """Counts the number of trainable parameters in a PyTorch model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [84]:


VanillaModels = [VanillaLSTM,VanillaBidirectionalRNN,VanillaBidirectionalLSTM]
# VanillaModels = [VanillaRNN]


for model_class in VanillaModels:
    print(f"\n--- Training {model_class.__name__} ---")
    model = model_class(embedding_matrix = embedding_matrix_tensor).to(device)
    model_name = model_class.__name__
    model_save_dir = os.path.join(model_save_base_dir, model_name)

    best_model , training_time_seconds = train_model(model,learning_rate,epochs,train_dataloader,val_dataloader,model_save_dir=model_save_dir)
    #
    # Call the function and get the results dictionary
    results= get_statistics(best_model, test_dataloader, 0.5)
   
    # Calculate trainable parameters
    num_trainable_params = count_parameters(best_model)
# Extract individual metrics from the results dictionary
    accuracy = results['accuracy']
    precision = results['precision']
    recall = results['recall']
    f1_score = results['f1_score']
    confusion_matrix_array = results['confusion_matrix'] # Get the NumPy array
    
    
    # Flatten the confusion matrix for logging (TN, FP, FN, TP)
    # Assuming binary classification with cm = [[TN, FP], [FN, TP]]
    cm_tn, cm_fp, cm_fn, cm_tp = confusion_matrix_array.ravel()

     # Prepare the data row for the log file
    log_line = (f"{model_name},"
                    f"{accuracy:.4f},"
                    f"{precision:.4f},"
                    f"{recall:.4f},"
                    f"{f1_score:.4f},"
                    f"{cm_tn},"
                    f"{cm_fp},"
                    f"{cm_fn},"
                    f"{cm_tp},"
                f"{training_time_seconds:.2f},"  # New: Training Time
                f"{num_trainable_params}\n")      # New: Trainable Parameters

 

    #Append to log file
    with open(results_log_file_path, 'a') as f_log:
        f_log.write(log_line)

    print(f"Logged results for {model_name} to {results_log_file_path}")
    print(f"--- Finished training and logging for model: {model_class.__name__} ---")


print("\nAll Vanilla models trained and results logged.")



--- Training VanillaLSTM ---
Model saving directory created/ensured: /kaggle/working/saved_models/VanillaLSTM
Epoch 1/30 | Train Loss: 0.4023
Epoch 1/30 | Val Loss: 0.3472
Validation loss improved. Current best: 0.3472 (Epoch 1).
Epoch 2/30 | Train Loss: 0.3254
Epoch 2/30 | Val Loss: 0.3271
Validation loss improved. Current best: 0.3271 (Epoch 2).
Epoch 3/30 | Train Loss: 0.2947
Epoch 3/30 | Val Loss: 0.3238
Validation loss improved. Current best: 0.3238 (Epoch 3).
Epoch 4/30 | Train Loss: 0.2723
Epoch 4/30 | Val Loss: 0.3348
Validation loss did not improve for 1 epoch(s). Best was 0.3238 at Epoch 3.
Epoch 5/30 | Train Loss: 0.2542
Epoch 5/30 | Val Loss: 0.3285
Validation loss did not improve for 2 epoch(s). Best was 0.3238 at Epoch 3.
Epoch 6/30 | Train Loss: 0.2359
Epoch 6/30 | Val Loss: 0.3314
Validation loss did not improve for 3 epoch(s). Best was 0.3238 at Epoch 3.
Epoch 7/30 | Train Loss: 0.2262
Epoch 7/30 | Val Loss: 0.3463
Validation loss did not improve for 4 epoch(s). Best 

['movie',
 'utterly',
 'boring',
 'complete',
 'waste',
 'time',
 'regret',
 'watching']

[3, 1091, 188, 385, 253, 10, 2193, 41]

0.5038464665412903
Positive


## For models with Attention

In [93]:
# # --- Loop for Attention-Based Models ---
print("\n--- Starting training and evaluation for Attention-Based Models ---")
attention_types = [BahdanauAttention, LuongDotProductAttention, LuongGeneralAttention, LuongConcatAttention]
# Ensure attention_kwargs match the attention_types and their constructors
attention_kwargs = [
    {"attention_hidden_dim": HIDDEN_DIM}, # Bahdanau
    {},                                   # LuongDotProduct
    {},                                   # LuongGeneral
    {"attention_hidden_dim": HIDDEN_DIM}  # LuongConcat
]
plug_and_play_attention_models = [
    VanillaRNNWithPlugAndPlayAttention,
    VanillaBidirectionalRNNWithPlugAndPlayAttention,
]

for pap_a_model in plug_and_play_attention_models:
    for attention_type, attention_kwarg in zip(attention_types, attention_kwargs):
        base_model_name = pap_a_model.__name__.replace("WithPlugAndPlayAttention", "")
        attn_name = attention_type.__name__.replace("Attention", "")
        full_model_name = f"{base_model_name}With{attn_name}Attention"
        
        print(f"\n--- Training {full_model_name} ---")

        # Instantiate model
        # return_attention_weights=False is default, good for training
        model = pap_a_model(
            embedding_matrix=embedding_matrix_tensor,
            attention_class=attention_type,
            attention_kwargs=attention_kwarg,
            hidden_dim=HIDDEN_DIM,
            freeze_embeddings=True
        ).to(device)
        num_trainable_params = count_parameters(best_model)
        # # Print model summary for each attention model combination
        # # Use a consistent dummy input for summary
        # dummy_input_ids_summary = torch.randint(0, VOCAB_SIZE, (BATCH_SIZE, MAX_LEN), dtype=torch.long).to(device)
        # dummy_attention_mask_summary = torch.ones((BATCH_SIZE, MAX_LEN), dtype=torch.long).to(device)
        # for i in range(BATCH_SIZE):
        #     rand_len = torch.randint(low=1, high=MAX_LEN + 1, size=(1,)).item()
        #     dummy_attention_mask_summary[i, rand_len:] = 0
        
        # print(f"\n--- Model Summary {full_model_name} --- with {attention_type.__name__}")
        # # Note: If your attention models return a tuple (output, attention_weights),
        # # torchinfo might need to know which element is the primary output.
        # # It usually handles this by taking the first element of a tuple.
        # print(summary(model, input_data=(dummy_input_ids_summary, dummy_attention_mask_summary), device=str(device)))

        model_save_dir = os.path.join(model_save_base_dir, full_model_name)

        # Train model
        best_model, training_time_seconds = train_model(model, learning_rate, epochs, train_dataloader, val_dataloader, model_save_dir=model_save_dir)
        
        # Get statistics
        results = get_statistics(best_model, test_dataloader, 0.5)
        accuracy = results['accuracy']
        precision = results['precision']
        recall = results['recall']
        f1_score = results['f1_score']
        confusion_matrix_array = results['confusion_matrix']
        cm_tn, cm_fp, cm_fn, cm_tp = confusion_matrix_array.ravel()

        # Prepare log line
        log_line = (f"{full_model_name},"
                    f"{accuracy:.4f},"
                    f"{precision:.4f},"
                    f"{recall:.4f},"
                    f"{f1_score:.4f},"
                    f"{cm_tn},"
                    f"{cm_fp},"
                    f"{cm_fn},"
                    f"{cm_tp},"
                    f"{training_time_seconds:.2f}," # New: Training Time
                    f"{num_trainable_params}\n")     # New: Trainable Parameters

        # Append to log file
        with open(results_log_file_path, 'a') as f_log:
            f_log.write(log_line)
        
        print(f"Logged results for {full_model_name} to {results_log_file_path}")
        print(f"--- Finished training and logging for model: {full_model_name} ---")

print("\nAll models trained and results logged.")
print(f"Final results available at: {results_log_file_path}")



--- Starting training and evaluation for Attention-Based Models ---

--- Training VanillaRNNWithBahdanauAttention ---
Model saving directory created/ensured: /kaggle/working/saved_models/VanillaRNNWithBahdanauAttention
Epoch 1/30 | Train Loss: 0.6240
Epoch 1/30 | Val Loss: 0.5591
Validation loss improved. Current best: 0.5591 (Epoch 1).
Epoch 2/30 | Train Loss: 0.5463
Epoch 2/30 | Val Loss: 0.6263
Validation loss did not improve for 1 epoch(s). Best was 0.5591 at Epoch 1.
Epoch 3/30 | Train Loss: 0.5511
Epoch 3/30 | Val Loss: 0.5926
Validation loss did not improve for 2 epoch(s). Best was 0.5591 at Epoch 1.
Epoch 4/30 | Train Loss: 0.5400
Epoch 4/30 | Val Loss: 0.4995
Validation loss improved. Current best: 0.4995 (Epoch 4).
Epoch 5/30 | Train Loss: 0.4946
Epoch 5/30 | Val Loss: 0.6960
Validation loss did not improve for 1 epoch(s). Best was 0.4995 at Epoch 4.
Epoch 6/30 | Train Loss: 0.4621
Epoch 6/30 | Val Loss: 0.4293
Validation loss improved. Current best: 0.4293 (Epoch 6).
Epoch 

In [94]:
! zip -r models_with_attention /kaggle/working/saved_models/VanillaRNNWithBahdanauAttention /kaggle/working/saved_models/VanillaRNNWithLuongConcatAttention /kaggle/working/saved_models/VanillaRNNWithLuongDotProductAttention /kaggle/working/saved_models/VanillaRNNWithLuongGeneralAttention /kaggle/working/saved_models/VanillaBidirectionalRNNWithBahdanauAttention /kaggle/working/saved_models/VanillaBidirectionalRNNWithLuongConcatAttention /kaggle/working/saved_models/VanillaBidirectionalRNNWithLuongDotProductAttention /kaggle/working/saved_models/VanillaBidirectionalRNNWithLuongGeneralAttention 


  adding: kaggle/working/saved_models/VanillaRNNWithBahdanauAttention/ (stored 0%)
  adding: kaggle/working/saved_models/VanillaRNNWithBahdanauAttention/training_log.csv (deflated 44%)
  adding: kaggle/working/saved_models/VanillaRNNWithBahdanauAttention/loss_plot.png (deflated 7%)
  adding: kaggle/working/saved_models/VanillaRNNWithBahdanauAttention/best_model.pth (deflated 7%)
  adding: kaggle/working/saved_models/VanillaRNNWithLuongConcatAttention/ (stored 0%)
  adding: kaggle/working/saved_models/VanillaRNNWithLuongConcatAttention/training_log.csv (deflated 46%)
  adding: kaggle/working/saved_models/VanillaRNNWithLuongConcatAttention/loss_plot.png (deflated 10%)
  adding: kaggle/working/saved_models/VanillaRNNWithLuongConcatAttention/best_model.pth (deflated 7%)
  adding: kaggle/working/saved_models/VanillaRNNWithLuongDotProductAttention/ (stored 0%)
  adding: kaggle/working/saved_models/VanillaRNNWithLuongDotProductAttention/training_log.csv (deflated 39%)
  adding: kaggle/working